<a href="https://colab.research.google.com/github/Anmol-Josan/Unsupervised-Learning-For-HR-Breast-Cancer-RNA-Sequencing/blob/main/Data_Loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import os
import tarfile
from io import BytesIO

In [2]:
files_to_fetch = [
    {
        "name": "GSE300475_RAW.tar",
        "size": "565.5 Mb",
        "download_url": "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE300475&format=file",
        "type": "TAR (of CSV, MTX, TSV)"
    },
    {
        "name": "GSE300475_feature_ref.xlsx",
        "size": "5.4 Kb",
        "download_url": "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE300nnn/GSE300475/suppl/GSE300475%5Ffeature%5Fref.xlsx",
        "type": "XLSX"
    }
]

In [3]:
download_dir = "../Data"
os.makedirs(download_dir, exist_ok=True)
print(f"Downloads will be saved in: {os.path.abspath(download_dir)}\n")

def download_file(url, filename, destination_folder):
    """
    Downloads a file from a given URL to a specified destination folder.
    """
    filepath = os.path.join(destination_folder, filename)
    print(f"Attempting to download {filename} from {url}...")
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()

        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Successfully downloaded {filename} to {filepath}")
        return filepath
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {filename}: {e}")
        return None

Downloads will be saved in: c:\Users\ajosan\Downloads\Unsupervised-Learning-For-HR-Breast-Cancer-RNA-Sequencing\Data



In [4]:
for file_info in files_to_fetch:
    filename = file_info["name"]
    url = file_info["download_url"]
    file_type = file_info["type"]

    downloaded_filepath = download_file(url, filename, download_dir)

        # If the file is a TAR archive, extract it and list the contents
    if downloaded_filepath and filename.endswith(".tar"):
        print(f"Extracting {filename}...\n")
        try:
            with tarfile.open(downloaded_filepath, "r") as tar:
                # List contents
                members = tar.getnames()
                print(f"Files contained in {filename}:")
                for member in members:
                    print(f" - {member}")

                # Extract to a subdirectory within download_dir
                extract_path = os.path.join(download_dir, filename.replace(".tar", ""))
                os.makedirs(extract_path, exist_ok=True)
                tar.extractall(path=extract_path)
                print(f"\nExtracted to: {extract_path}")
        except tarfile.TarError as e:
            print(f"Error extracting {filename}: {e}")

        print("-" * 50 + "\n")

Attempting to download GSE300475_RAW.tar from https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE300475&format=file...
Successfully downloaded GSE300475_RAW.tar to Data\GSE300475_RAW.tar
Extracting GSE300475_RAW.tar...

Files contained in GSE300475_RAW.tar:
 - GSM9061665_S1_barcodes.tsv.gz
 - GSM9061665_S1_features.tsv.gz
 - GSM9061665_S1_matrix.mtx.gz
 - GSM9061666_S2_barcodes.tsv.gz
 - GSM9061666_S2_features.tsv.gz
 - GSM9061666_S2_matrix.mtx.gz
 - GSM9061667_S3_barcodes.tsv.gz
 - GSM9061667_S3_features.tsv.gz
 - GSM9061667_S3_matrix.mtx.gz
 - GSM9061668_S4_barcodes.tsv.gz
 - GSM9061668_S4_features.tsv.gz
 - GSM9061668_S4_matrix.mtx.gz
 - GSM9061669_S5_barcodes.tsv.gz
 - GSM9061669_S5_features.tsv.gz
 - GSM9061669_S5_matrix.mtx.gz
 - GSM9061670_S6_barcodes.tsv.gz
 - GSM9061670_S6_features.tsv.gz
 - GSM9061670_S6_matrix.mtx.gz
 - GSM9061671_S7_barcodes.tsv.gz
 - GSM9061671_S7_features.tsv.gz
 - GSM9061671_S7_matrix.mtx.gz
 - GSM9061672_S8_barcodes.tsv.gz
 - GSM9061672_S8_features.tsv.g

C:\Users\ajosan\AppData\Local\Temp\ipykernel_22488\128555780.py:22: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_path)



Extracted to: Data\GSE300475_RAW
--------------------------------------------------

Attempting to download GSE300475_feature_ref.xlsx from https://ftp.ncbi.nlm.nih.gov/geo/series/GSE300nnn/GSE300475/suppl/GSE300475%5Ffeature%5Fref.xlsx...
Successfully downloaded GSE300475_feature_ref.xlsx to Data\GSE300475_feature_ref.xlsx


In [5]:
import gzip
import shutil
from pathlib import Path
import pandas as pd
from scipy.io import mmread

def decompress_gz_file(gz_path, output_dir):
    """
    Decompress a .gz file to the specified output directory.
    """
    output_path = os.path.join(output_dir, Path(gz_path).stem)
    print(f"Decompressing {gz_path} → {output_path}")
    try:
        with gzip.open(gz_path, 'rb') as f_in, open(output_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
        return output_path
    except Exception as e:
        print(f"Failed to decompress {gz_path}: {e}")
        return None

def preview_file(file_path):
    """
    Display the first few lines of a decompressed file, based on its extension.
    """
    print(f"\n--- Preview of {os.path.basename(file_path)} ---")
    try:
        if file_path.endswith(".tsv"):
            df = pd.read_csv(file_path, sep='\t')
            print(df.head())
        elif file_path.endswith(".csv"):
            df = pd.read_csv(file_path)
            print(df.head())
        elif file_path.endswith(".mtx"):
          matrix = mmread(file_path).tocoo()
          print("First 5 non-zero entries:")
          for i in range(min(5, len(matrix.data))):
              print(f"Row: {matrix.row[i]}, Col: {matrix.col[i]}, Value: {matrix.data[i]}")
          print(f"\nMatrix shape: {matrix.shape}, NNZ (non-zero elements): {matrix.nnz}")
        else:
            print("Unsupported file type for preview.")
    except Exception as e:
        print(f"Could not preview {file_path}: {e}")

extract_dir = os.path.join(download_dir, "GSE300475_RAW")

for root, _, files in os.walk(extract_dir):
    for file in files:
        if file.endswith(".gz"):
            gz_file_path = os.path.join(root, file)
            decompressed_path = decompress_gz_file(gz_file_path, root)
            if decompressed_path:
                preview_file(decompressed_path)

Decompressing Data\GSE300475_RAW\GSM9061665_S1_barcodes.tsv.gz → Data\GSE300475_RAW\GSM9061665_S1_barcodes.tsv

--- Preview of GSM9061665_S1_barcodes.tsv ---
   AAACCTGAGAAGGGTA-1
0  AAACCTGAGACTGTAA-1
1  AAACCTGAGCAGCGTA-1
2  AAACCTGAGCCAACAG-1
3  AAACCTGAGCGTGAAC-1
4  AAACCTGAGCTACCTA-1
Decompressing Data\GSE300475_RAW\GSM9061665_S1_features.tsv.gz → Data\GSE300475_RAW\GSM9061665_S1_features.tsv

--- Preview of GSM9061665_S1_features.tsv ---
   ENSG00000243485 MIR1302-2HG  Gene Expression
0  ENSG00000237613     FAM138A  Gene Expression
1  ENSG00000186092       OR4F5  Gene Expression
2  ENSG00000238009  AL627309.1  Gene Expression
3  ENSG00000239945  AL627309.3  Gene Expression
4  ENSG00000239906  AL627309.2  Gene Expression
Decompressing Data\GSE300475_RAW\GSM9061665_S1_matrix.mtx.gz → Data\GSE300475_RAW\GSM9061665_S1_matrix.mtx

--- Preview of GSM9061665_S1_matrix.mtx ---
First 5 non-zero entries:
Row: 24, Col: 0, Value: 1
Row: 104, Col: 0, Value: 1
Row: 145, Col: 0, Value: 1
Row: 1

In [6]:
import glob

# Find all "all_contig_annotations.csv" files in the extracted directory and sum their lengths (number of rows)

all_contig_files = glob.glob(os.path.join(extract_dir, "*_all_contig_annotations.csv"))
total_rows = 0

for file in all_contig_files:
    try:
        df = pd.read_csv(file)
        num_rows = len(df)
        print(f"{os.path.basename(file)}: {num_rows} rows")
        total_rows += num_rows
    except Exception as e:
        print(f"Could not read {file}: {e}")

print(f"\nTotal rows in all contig annotation files: {total_rows}")

GSM9061687_S1_all_contig_annotations.csv: 16782 rows
GSM9061688_S2_all_contig_annotations.csv: 24445 rows
GSM9061689_S3_all_contig_annotations.csv: 18336 rows
GSM9061690_S4_all_contig_annotations.csv: 12694 rows
GSM9061691_S5_all_contig_annotations.csv: 4074 rows
GSM9061692_S6_all_contig_annotations.csv: 18505 rows
GSM9061693_S7_all_contig_annotations.csv: 17601 rows
GSM9061694_S9_all_contig_annotations.csv: 18147 rows
GSM9061695_S10_all_contig_annotations.csv: 17759 rows
GSM9061696_S11_all_contig_annotations.csv: 13790 rows

Total rows in all contig annotation files: 162133
